In [29]:
import requests
import pandas as pd
import sqlite3


In [30]:
url = 'https://brasilapi.com.br/api/cvm/corretoras/v1'
response = requests.get(url)

In [31]:
if response.status_code == 200:
    print('Dados importados com sucesso')
    data_json = response.json()
else: 
    print('Erro ao acessar a API')

Dados importados com sucesso


In [32]:
corretoras = pd.DataFrame(data_json)

In [33]:
corretoras_cadastro = corretoras[['cnpj', 'nome_social', 'nome_comercial', 'codigo_cvm', 'email', 'telefone', 'logradouro', 'complemento', 'cep']]

corretoras_cadastro = corretoras_cadastro.rename(columns={
    'cnpj': 'CNPJ', 
    'nome_social': "Nome", 
    'nome_comercial': 'Nome Comercial', 
    'codigo_cvm': 'Código CVM',
    'email': 'E-mail', 
    'telefone': 'Telefone', 
    'logradouro': 'Endereço', 
    'complemento': 'Complemento', 
    'cep': 'CEP'})

corretoras_cadastro = corretoras_cadastro.set_index(['CNPJ', 'Nome'])
corretoras_cadastro_stacked = corretoras_cadastro.stack()

corretoras_cadastro_stacked


CNPJ            Nome                                       
76621457000185  4UM DTVM S.A.                Nome Comercial                 4UM INVESTIMENTOS
                                             Código CVM                                  2275
                                             E-mail                       controle@4um.com.br
                                             Telefone                                33519966
                                             Endereço          R. VISCONDE DO RIO BRANCO 1488
                                                                            ...              
02332886000104  XP INVESTIMENTOS CCTVM S.A.  E-mail                lucas.rabechini@xpi.com.br
                                             Telefone                                30272237
                                             Endereço            AVENIDA ATAULFO DE PAIVA 153
                                             Complemento                             5º Andar


In [38]:
conn = sqlite3.connect('coderhouse.db')
corretoras_cadastro_stacked.to_sql('corretoras_cadastro', conn, if_exists='replace', index=False)
conn.close()


In [47]:
corretoras_status = corretoras[['cnpj', 'data_registro', 'status']]

corretoras_status = corretoras_status.rename(columns={
    'cnpj': 'CNPJ', 
    'data_registro': 'Data de Registro', 
    'status': 'Status'})

corretoras_status_replace = corretoras_status.replace(to_replace='EM FUNCIONAMENTO NORMAL',
                                                    value = 'EM FUNCIONAMENTO')

corretoras_status_replace['Data de Registro'] = pd.to_datetime(corretoras_status_replace['Data de Registro'],format='%Y-%m-%d')

corretoras_status_replace.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373 entries, 0 to 372
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   CNPJ              373 non-null    object        
 1   Data de Registro  373 non-null    datetime64[ns]
 2   Status            373 non-null    object        
dtypes: datetime64[ns](1), object(2)
memory usage: 8.9+ KB


In [42]:
conn = sqlite3.connect('coderhouse.db')
corretoras_status.to_sql('corretoras_status', conn, if_exists='replace', index=False)
conn.close()

In [45]:
corretoras_patrimonio = corretoras[['cnpj', 'valor_patrimonio_liquido']]

corretoras_patrimonio = corretoras_patrimonio.rename(columns={
    'cnpj': 'CNPJ',
    'valor_patrimonio_liquido': 'Patrimônio Líquido'})



corretoras_patrimonio['Patrimônio Líquido'] = pd.to_numeric(corretoras_patrimonio['Patrimônio Líquido'])

filtro = corretoras_patrimonio['Patrimônio Líquido'] > 3000000

corretoras_patrimonio_filtrado = corretoras_patrimonio[filtro]

corretoras_patrimonio_filtrado


,CNPJ,Patrimônio Líquido
0,76621457000185,4.228660e+06
2,10664027000132,5.995252e+06
3,44527444000155,3.729687e+06
7,43050917000103,8.518212e+06
12,74014747000135,4.498104e+08
...,...,...
367,36159085000100,1.684848e+07
368,01170892000131,2.667918e+08
369,61769790000169,7.464374e+06
370,92875780000131,2.339812e+07


In [46]:
conn = sqlite3.connect('coderhouse.db')
corretoras_patrimonio_filtrado.to_sql('corretoras_patrimonio', conn, if_exists='replace', index=False)
conn.close()